In [3]:
import pandapower as pp
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler
import pickle
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import torch
from stable_baselines3 import PPO

ModuleNotFoundError: No module named 'gymnasium'

In [ ]:
class SimpleTwoBus:
 
    def __init__(self, V_ext, P, Q, G, B, V_init, theta_init):
        '''This class creates a simple 2-bus network.'''
        
        self.V_ext = V_ext
        self.P = P
        self.Q = Q
        self.G = G
        self.B = B
        self.V_init = V_init
        self.theta_init = theta_init
        self.net = pp.create_empty_network()
        self.create_two_bus_grid()
 
 
    def create_two_bus_grid(self):
   
        # Create two buses with initialized voltage and angle
        bus1 = pp.create_bus(self.net, vn_kv=20.0, name="Bus 1")
        bus2 = pp.create_bus(self.net, vn_kv=0.4, name="Bus 2")
   
        # Initialize voltage and angle for buses
        self.net.bus.loc[bus1, 'vm_pu'] = self.V_init[0]
        self.net.bus.loc[bus1, 'va_degree'] = self.theta_init[0]
        self.net.bus.loc[bus2, 'vm_pu'] = self.V_init[1]
        self.net.bus.loc[bus2, 'va_degree'] = self.theta_init[1]
   
        # create a line between the two buses
        pp.create_line_from_parameters(
            self.net,
            from_bus=0,
            to_bus=1,
            length_km=1.0,
            r_ohm_per_km=1/self.G,
            x_ohm_per_km=1/self.B,
            c_nf_per_km=0.0,
            g_us_per_km=0.0,
            max_i_ka=100.0,
        )
 
        # Create a transformer between the two buses
        # pp.create_transformer(self.net, bus1, bus2, std_type="0.25 MVA 20/0.4 kV")
   
        # Create a load at bus 2 with specified P and Q
        pp.create_load(self.net, bus2, p_mw=self.P, q_mvar=self.Q, name="Load")
   
        # Create an external grid connection at bus 1 with specified G and B
        pp.create_ext_grid(self.net, bus1, vm_pu=self.V_ext, name="Grid Connection")


In [ ]:
class GridEnv(gym.Env):
    def __init__(self,V_ext = 1.02, G = 100, B = 0.1, k_limit = 5, termination_counter=100):

        self.observation_space = spaces.Box(low = np.array([-1e10,-1e10]), high = np.array([1e10, 1e10]), dtype=np.float32)
        self.action_space = spaces.Box(low=np.array([-0.05, -30]), high=np.array([0.05, 30]), dtype=np.float32)

        self.k_limit = k_limit
        self.termination_counter = termination_counter


        self.G = G
        self.B = B
        self.V_ext = V_ext

        #initialize network
        self.reset()
    
    
    def reset(self):

        self.counter = 0
        self.done = False
        self.terminated = False

        self.P = np.random.uniform(low= 0, high=0.2)
        self.Q = np.random.uniform(low = 0, high = 0.1)
        self.V_init = np.random.uniform(low = 0.9, high = 1.1, size=2)
        self.theta_init = np.random.uniform(low = -20, high = 20, size=2)

        Net = SimpleTwoBus(self.V_ext,self.P,self.Q,self.G,self.B,self.V_init,self.theta_init)
        self.net = Net.net

        initial_guesses = np.array([self.V_init, self.theta_init])

        self.state = self.calculate_residual(initial_guesses)

    
 
    # def compute_residual_torch(self, V_mag, V_ang, Ybus, S):
    #     V_ang = torch.deg2rad(V_ang)
    #     complex_v = V_mag*(torch.exp(V_ang*1j))
    #     current = Ybus@complex_v
    #     diag_V = torch.diag(complex_v)
    #     residual = diag_V@torch.conj(current) - S
    #     return residual 


    def calculate_residual(self, action):

        net = self.net.deepcopy()  # Keep the network unchanged

        pp.runpp(net, max_iteration = 1, tolerance_mva = np.inf) # not the correct function, this is just to let the environment loop be able to run
        err = net._ppc['et']

        residual = np.zeros(2)
        residual[:] = err
        
        #needs a function!

        return residual


    def perform_NR_step(self):

        net = self.net.deepcopy()  # Keep the network unchanged

        pp.runpp(net, max_iteration = 50, tolerance_mva = 1e-5)

        iterations = net._ppc["iterations"]

        return iterations
        


    def calculate_reward(self):

        iterations = self.perform_NR_step()

        reward = - iterations

        return reward
    

    def step(self, action):



        # action = [delta_V, delta_theta]

        # perform action
        residual = self.calculate_residual(action)


        # calcualate reward
        reward = self.calculate_reward()


        #update state:
        self.state = residual

        if reward == -self.k_limit:
            self.done = True

        self.counter += 1

        if self.counter == self.termination_counter:
            self.terminated = True

        return self.state, reward, self.done, self.terminated


  

    def render(self):
        pass

In [ ]:
# Test run


env = GridEnv()


state = env.reset()
print("Initial State:")
# env.render()

# Define a sample action within the specified ranges
action = np.array([0.03, 15.0], dtype=np.float32)

# Take a step in the environment using the sample action
next_state, reward, done, info = env.step(action)

# Print the results
print("\nAction Taken:", action)
print("Next State:", next_state)
# env.render()
print("Reward:", reward)
print("Done:", done)

In [ ]:
# train an RL agent on the environment from above

env = GridEnv()
lr = 3e-3
total_timesteps = 1e6

model = PPO("MlpPolicy", env, verbose=1, learning_rate = lr)
model.learn(total_timesteps=total_timesteps)
# model.save("")

# del model # remove to demonstrate saving and loading

# model = PPO.load("ppo_cartpole")

# obs = vec_env.reset()
# while True:
#     action, _states = model.predict(obs)
#     obs, rewards, dones, info = vec_env.step(action)
#     vec_env.render("human")


In [ ]:
def evaluate_model(model, num_evaluations, env):
    # idea: plot residual as a function of actions?
    raise NotImplementedError